# Database models

> This module contains ORM models for the project entities

## Implementation

In [ ]:
#| default_exp models

In [ ]:
#| export
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, Text, String, JSON, DateTime, ForeignKey, UniqueConstraint, PrimaryKeyConstraint, \
    create_engine
from sqlalchemy.ext.asyncio import create_async_engine
from sqlalchemy.orm import relationship
from pino_inferior.core import SQLALCHEMY_CONNECTION_STRING

In [ ]:
#| export
aengine = create_async_engine(url=SQLALCHEMY_CONNECTION_STRING)

In [ ]:
#| export
Base = declarative_base()

C:\Users\alex4321\AppData\Local\Temp\ipykernel_18764\3325957931.py:2: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [ ]:
#| export
class ParagraphMemoryRecord(Base):
    """
    Memory system operates on document paragraph level in SQL database and sentences in vector DB.
    So:
    1. query is converted to an embedding
    2. than vector DB finds most similar sentences
    3. than corresponding paragraph extracted from SQL database
    And vice versa for storing.
    This ORM entity related to the last entity (SQL database records regards paragraphs).
    """
    __tablename__ = "memory_records"
    id = Column(Integer, primary_key=True, autoincrement=True)
    text = Column(Text)
    meta = Column(JSON)
    md5 = Column(String(64))
    created_at = Column(DateTime)

    __table_args__ = (
        UniqueConstraint('md5', 'text', name='_md5_text_uc'),
    )

In [ ]:
#| export
class APICommentQuery(Base):
    __tablename__ = "comment_query"
    query_id = Column(Integer, primary_key=True, autoincrement=True)
    
    time = Column(DateTime)
    context = Column(Text)
    history = Column(JSON)

    user_name = Column(Text)
    user_character = Column(Text)
    user_goals = Column(Text)
    user_style_example = Column(Text)
    user_style_description = Column(Text)
    #AGENT_INPUT_FALLACIES: read_fallacies(FALLACIES_FNAME),
    #AGENT_INPUT_TOOLS: [fallacy_tool, memory_tool,],
    

In [ ]:
#| export
class APIContextSummarizationQuery(Base):
    __tablename__ = "context_summary_query"
    query_id = Column(Integer, primary_key=True, autoincrement=True)
    text = Column(Text)
    time = Column(DateTime)
    post_time = Column(DateTime)

    user_name = Column(Text)
    user_character = Column(Text)
    user_goals = Column(Text)

In [ ]:
#| export
class APITask(Base):
    __tablename__ = "api_task"
    task_id = Column(Integer, primary_key=True, autoincrement=True)
    created_at = Column(DateTime)
    updated_at = Column(DateTime)
    status = Column(Integer)
    response = Column(Text)

In [ ]:
#| export
class APICommentQueryTaskMapping(Base):
    __tablename__ = "api_comment_task"
    comment_id = Column(Integer, ForeignKey(f"{APICommentQuery.__tablename__}.query_id"))
    task_id = Column(Integer, ForeignKey(f"{APITask.__tablename__}.task_id"))

    __table_args__ = (
        PrimaryKeyConstraint(
            "comment_id",
            "task_id",
            name="pk_cid_tid"
        ),
        UniqueConstraint(
            "task_id",
            name="unique_tid",
        ),
    )

In [ ]:
#| export
class APIContextSummarizationTaskMapping(Base):
    __tablename__ = "api_context_task"
    context_id = Column(Integer, ForeignKey(f"{APIContextSummarizationQuery.__tablename__}.query_id"))
    task_id = Column(Integer, ForeignKey(f"{APITask.__tablename__}.task_id"))
    
    __table_args__ = (
        PrimaryKeyConstraint(
            "context_id",
            "task_id",
            name="pk_cid_tid"
        ),
        UniqueConstraint(
            "task_id",
            name="unique_tid",
        ),
    )

In [ ]:
import nbdev; nbdev.nbdev_export()